### Import libraries

Import libraries, set parameters, and import classes

In [ ]:
import numpy as np
import tensorflow as tf
import pickle
np.random.seed(1)
tf.random.set_seed(1)
tf.keras.backend.set_floatx('float32')

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (3,3)
plt.rcParams.update({'font.size': 13})

In [ ]:
from __lib__.NN_class import NeuralNetwork
from __lib__.DataProcessing import PreProcessing

### Import datasets and pre-processing

In [ ]:
file = './data/3D_PlaneStrain_VonMises_CSMA'
training_percentage = 0.6

In [ ]:
with open(file, 'rb') as f_obj:
    data = pickle.load(f_obj)

strain,strainDot,svars,svarsDot,stress,energy,dissipationDot = data

del data


##### Split datasets  (tv=training+validation - test)

In [ ]:
PreProcessing = PreProcessing(training_percentage)
train_p,val_p,test_p = PreProcessing.split_dataset(strain,percentage=True)

print('# samples:', train_p+val_p+test_p,
      '\n# training:', train_p,
      '\n# validation:', val_p,
      '\n# test:', test_p)

In [ ]:
strain_tv,strain_test = PreProcessing.split_dataset(strain)
strainDot_tv,strainDot_test = PreProcessing.split_dataset(strainDot)
svars_tv,svars_test = PreProcessing.split_dataset(svars)
svarsDot_tv,svarsDot_test = PreProcessing.split_dataset(svarsDot)
stress_tv,stress_test = PreProcessing.split_dataset(stress)

##### Get normalization parameters

In [ ]:
# Use simply the code you programmed previously for the elastic case

In [ ]:
# Or:
# prm_E = PreProcessing.GetParams(strain_tv,normalize=True)
# prm_EDot = PreProcessing.GetParams(strainDot_tv,normalize=True)
# prm_Z = PreProcessing.GetParams(svars_tv,normalize=True)
# prm_ZDot = PreProcessing.GetParams(svarsDot_tv,normalize=True)
# prm_S = PreProcessing.GetParams(stress_tv,normalize=False)

##### Normalize datasets

In [ ]:
# Training and validation datasets

__strain_tv,__strain_test = PreProcessing.Normalize(
    [strain_tv,strain_test],prm_E,train_val=True)
__strainDot_tv,__strainDot_test = PreProcessing.Normalize(
    [strainDot_tv,strainDot_test],prm_EDot,train_val=True)
__svars_tv,__svars_test = PreProcessing.Normalize(
    [svars_tv,svars_test],prm_Z,train_val=True)
__svarsDot_tv,__svarsDot_test = PreProcessing.Normalize(
    [svarsDot_tv,svarsDot_test],prm_ZDot,train_val=True)
__stress_tv,__stress_test = PreProcessing.Normalize(
    [stress_tv,stress_test],prm_S,train_val=True)



### Neural Networks

#### Build network

In [ ]:
umat_dim = 4 # dimensional space

hidden_NN_Stress = 3 # number of hidden layers in NN Evolution
hidden_NN_Evolution = 3
material = 'inelastic'

Nnet = NeuralNetwork(umat_dim,
                     material,
                     hidden_NN_Stress,
                     activation_NN_Stress='leaky_relu',
                     hidden_NN_Evolution = hidden_NN_Evolution,
                     activation_NN_Evolution='leaky_relu',
                    ) # Initialize TANN class


params = [prm_E,prm_S,prm_Z,prm_EDot,prm_ZDot] # regroup normalization parameters

Nnet.SetParams(params) # set normalization parameters in the model

# build TF model class
input_shape = [[None,umat_dim],[None,umat_dim],[None,umat_dim]]
Nnet.build(input_shape)

# print model's summary
print(Nnet.summary())

#### Define inputs and outputs

In [ ]:
output_tv = [__svarsDot_tv,__stress_tv]

output_test = [__svarsDot_test,__stress_test]

input_tv = [__svars_tv,__strain_tv,__strainDot_tv]

input_test = [__svars_test,__strain_test,__strainDot_test]

#### Compile and train the network

In [ ]:
Nnet.compileNet()

callbacks_list = [tf.keras.callbacks.EarlyStopping(monitor ='val_loss',min_delta = 1.e-6,
                                                   patience = 200, verbose = 0, mode = 'min',
                                                   baseline = None,restore_best_weights = True)]

hist_Nnet = Nnet.fit(input_tv,output_tv,
                     epochs = 1500,
                     validation_split=0.2,
                     batch_size = 200,
                     verbose = 2,
                     use_multiprocessing=True,
                     callbacks = callbacks_list,)

#### Evaluate losses

In [ ]:
print('Evaluate loss for training and validation datasets:'); Nnet.evaluate(input_tv,output_tv,verbose=2)
print('\nEvaluate loss for test datasets:'); Nnet.evaluate(input_test,output_test,verbose=2)


svarsDot_loss = np.asarray(hist_Nnet.history['output_1_loss'])
svarsDot_val_loss = np.asarray(hist_Nnet.history['val_output_1_loss'])
stress_loss = np.asarray(hist_Nnet.history['output_2_loss'])
stress_val_loss = np.asarray(hist_Nnet.history['val_output_2_loss'])
epochs = range(1, len(stress_loss) + 1)

#### Plot training curves

In [ ]:
fig, ax = plt.subplots(1)
ax.loglog(epochs, svarsDot_loss, color='red', label='$\dot{z}$', linewidth=2, marker='o',markersize=0)
ax.loglog(epochs, svarsDot_val_loss, color='red', alpha=0.5,  linewidth=2, marker='o',markersize=0)

ax.loglog(epochs, stress_loss, color='black', label='$\sigma$', linewidth=2, marker='o',markersize=0)
ax.loglog(epochs, stress_val_loss, color='black', alpha=0.5,  linewidth=2, marker='o',markersize=0)


ax.set_xlabel('epochs')
ax.set_ylabel('Mean Absolute Error')
ax.legend(loc='best')
ax.grid()
ax.set_ylim(1e-7,1)

#### Predictions for the entire batch

In [ ]:
pred__svarsDot,pred__stress=Nnet.predict(input_test)

# Plot stress
plt.plot(__stress_test,pred__stress,
         alpha=0.3,linewidth=0, marker='o',markersize=5)
plt.xlabel("$\sigma$")
plt.ylabel("$\sigma=NN(\\varepsilon)$")
plt.show()

# Plot internal variable
plt.plot(__svarsDot_test,
         pred__svarsDot,
         alpha=0.3,linewidth=0, marker='o',markersize=5)
plt.xlabel("$\sigma$")
plt.ylabel("$\sigma=\partial_{\\varepsilon} NN_{\psi}(\\varepsilon)$")
plt.show()

### Save model

In [ ]:
# filename = './results/NN_VonMises'
# Nnet.save_export(filename)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b33f54f1-f98d-4fed-99c0-2484f115edd9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>